In [1]:
import pandas
import mmengine
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')

In [2]:
from projects.detr3d.custom_concat_dataset import CustomNusc
from projects.configs import debug
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend

In [8]:
register_all_modules()
cfg = copy.deepcopy(debug.nusc_val)
cfg['ann_file'] = 'nuscenes_infos_trainval.pkl'
cfg['data_root'] = 'data/nus_v2/'
cfg.pop('type')
cfg.pop('load_interval',None)
dataset = CustomNusc(**cfg)

04/16 11:32:03 - mmengine - INFO - ------------------------------
04/16 11:32:03 - mmengine - INFO - The length of the dataset: 34149
04/16 11:32:03 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 493322 |
| truck                | 88519  |
| construction_vehicle | 14671  |
| bus                  | 16321  |
| trailer              | 24860  |
| motorcycle           | 12617  |
| bicycle              | 11859  |
| pedestrian           | 220194 |
+----------------------+--------+


In [3]:
register_all_modules()
cfg = copy.deepcopy(debug.nusc_val)
cfg['ann_file'] = 'nuscenes_infos_trainval.pkl'
cfg['data_root'] = 'data/nus_v2/'
cfg.pop('type')
cfg.pop('load_interval',None)
dataset = CustomNusc(**cfg)

04/16 11:06:37 - mmengine - INFO - ------------------------------
04/16 11:06:37 - mmengine - INFO - The length of the dataset: 34149
04/16 11:06:37 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 493322 |
| truck                | 88519  |
| construction_vehicle | 14671  |
| bus                  | 16321  |
| trailer              | 24860  |
| motorcycle           | 12617  |
| bicycle              | 11859  |
| pedestrian           | 220194 |
+----------------------+--------+


In [3]:
from nuscenes import NuScenes
nusc = NuScenes('v1.0-trainval','data/nuscenes/')

Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 lidarseg,
Done loading in 83.588 seconds.
Reverse indexing ...
Done reverse indexing in 20.6 seconds.


In [4]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
import os.path as osp
import os
import numpy as np
from projects.detr3d.detr3d import DETR3D
import torch
from mmdet3d.structures import LiDARInstance3DBoxes
from mmengine.structures import InstanceData
detr3d = DETR3D()
token2idx = pandas.read_pickle('debug/lyc_map/token2idx_trainval.pkl')
vis = visualizer_zlt(debug_name='',vis_count=10000, draw_score_type = False)


In [5]:
def view_frame(token, map_points):
    idx = token2idx[token]
    frame = dataset[idx]
    kpts= map_points
    trans = [ 646.2525, 1612.6713,    1.8371]# should be lidar2ego but not 2global
    # kpts = np.load('/home/zhenglt/mmdev11/detr3d-dev11/debug/0a0d6b8c2e884134a3b48df43d54c36a.npy')
    kpts_3d = np.zeros((kpts.shape[0],7))
    gt_labels_3d = torch.zeros((kpts.shape[0]))
    if kpts.shape[-1] == 2:
        kpts_3d[:,:2] = kpts
        kpts_3d[:,2] -= trans[-1]
    else:
        kpts_3d[:,:3] = kpts
    gt_bboxes_3d = LiDARInstance3DBoxes(torch.tensor(kpts_3d),box_dim=7,origin=(0.5,0.5,0.5))
    map_inst = vis.toInstance({'gt_bboxes_3d': gt_bboxes_3d, 'gt_labels_3d': gt_labels_3d}, device=gt_labels_3d.device)
    name = osp.basename(frame['data_samples'].lidar_path).split('.')[0]
    inst = frame['data_samples'].gt_instances_3d[:2]
    gt_bboxes_3d = inst.bboxes_3d
    print(name)
    # vis.visualize(inst, batch_input_metas,None,'test','debug/')
    cat_instances = InstanceData.cat([map_inst, vis.add_score(frame['data_samples'].gt_instances_3d)])
    if not osp.exists('debug/lyc_map/'+token):
        os.mkdir('debug/lyc_map/'+token)
    vis.visualize_dataset_item(frame,[cat_instances],pts = None,name_suffix='debug_pred', dirname='debug/lyc_map/'+token)

In [6]:
RANGE = 102.4
def filter_pts_inrange(pts):
        max_x = RANGE / 2
        max_y = RANGE / 2
        mask = (-max_x < pts[...,0]) & (pts[..., 0] < max_x) & \
               (-max_y < pts[...,1]) & (pts[..., 1] < max_y)
        pts = pts[mask]
        # return none if nothing
        return pts

In [23]:
from projects.detr3d.detr3d_featsampler import DefaultFeatSampler
import mmengine
feat_samp = DefaultFeatSampler()
depths = [[],[],[],[],[],[]]
for i in range(0,len(dataset),5):
    frame = dataset[i]
    pts = torch.tensor(np.fromfile(frame['data_samples'].metainfo['lidar_path'], dtype=np.float32).reshape(-1,5))[:,:3]
    pts = filter_pts_inrange(pts)
    batch_input_metas = [frame['data_samples'].metainfo]
    batch_input_metas = vis.add_lidar2img(batch_input_metas)
    batch_input_metas[0]['pad_shape'] = 900, 1600
    pt_cam, mask, depth= feat_samp.project_ego2cam(pts.view(1, -1, 3),debug.point_cloud_range, batch_input_metas, return_depth=True)
    pt_cam = pt_cam.squeeze(0)
    mask = mask.squeeze(0).squeeze(-1)  # [cam, gt]
    depth = depth.squeeze(0)
    pts = torch.cat((pts, torch.ones_like(pts[..., :1])), -1)
    for cam in range(depth.shape[0]):
        # depth_i = depth[cam][mask[cam]]
        lidar2cam_i = batch_input_metas[0]['lidar2cam'][cam]
        pts_cami = torch.matmul(torch.tensor(lidar2cam_i), pts.T)
        depth_i = pts_cami[2][mask[cam]]
        # print(depth_i.shape)
        # TODO: debug why there are so many negetive points
        depths[cam].append(depth_i)
mmengine.dump(depths, 'debug/depth_stat/depths.pkl')
cam_name = list(debug.nusc_data_prefix.keys())
cam_name.remove('pts')
cam_name.remove('sweeps')
import matplotlib.pyplot as plt
for depth,name in zip(depths,cam_name):
    depth = torch.cat(depth) 
    depth = np.array(depth)
    plt.hist(depth,bins = 10)
    plt.savefig('debug/depth_stat/'+name)
    print('save to '+'debug/depth_stat/'+name)
    plt.gcf().clear()

save to debug/depth_stat/CAM_FRONT
save to debug/depth_stat/CAM_FRONT_RIGHT
save to debug/depth_stat/CAM_FRONT_LEFT
save to debug/depth_stat/CAM_BACK
save to debug/depth_stat/CAM_BACK_LEFT
save to debug/depth_stat/CAM_BACK_RIGHT


<Figure size 640x480 with 0 Axes>

In [42]:

nusc_map = VectorizeLocalMap(data_root='data/nuscenes/')
map_data = pandas.read_pickle('debug/lyc_map/map_data_upsampling.pkl')
for token in map_data:
    idx = token2idx[token]
    if token != 'a08cf86f5b2a4f5abdee5756820aa66f':
        continue
    frame = dataset[idx]
    location = frame['data_samples'].metainfo['city_name']
    # patch_box = 
    from pyquaternion import Quaternion
    e2g = np.array(frame['data_samples'].metainfo['ego2global'])
    rot = Quaternion(matrix=e2g[:3,:3],atol=1e-5)
    yaw = rot.yaw_pitch_roll[0] /np.pi *180
    center_xy = e2g[:2,3]
    size_xy = [100,100]
    gt_list = nusc_map(((*center_xy,*size_xy), yaw, location))
    l2e = np.array(frame['data_samples'].metainfo['lidar2ego'])
    e2l = torch.tensor(np.linalg.inv(l2e))
    map_points = []
    for gt in gt_list:
        map_points.append(torch.tensor(gt))
    map_points = torch.cat(map_points)
    map_z = torch.zeros_like(map_points[:,0:1])
    map_t = torch.ones_like(map_points[:,0:1])
    map_points = torch.cat([map_points, map_z, map_t],dim=-1)
    e2g_conpensate = Quaternion(axis=[0, 1, 0], radians=rot.yaw_pitch_roll[1]) * Quaternion(axis=[1, 0, 0], radians=rot.yaw_pitch_roll[2])
    e2g_R = e2g_conpensate.rotation_matrix
    g2e_R = np.linalg.inv(e2g_R)
    g2e_R4d = np.identity(4)
    g2e_R4d[:3,:3] = g2e_R
    map_points = torch.matmul(torch.tensor(g2e_R4d),map_points.T).T
    map_points = torch.matmul(e2l,map_points.T).T
    # print(filter_pts_inrange(map_points))
    # print(map_points[:,:3])
    # break
    view_frame(token, map_points[:,:3])
    # break
    # print(np.array(Quaternion(matrix=l2e[:3,:3],atol=1e-5).yaw_pitch_roll)/np.pi*180)
    # print(np.array(rot.yaw_pitch_roll)/np.pi*180)
    # break
    

# lidar_token = nusc.get('sample',token)['data']['LIDAR_TOP']
# ego = nusc.get('sample_data',lidar_token)['ego_pose_token']
# ego_pose = nusc.get('ego_pose',ego)
# nusc_e2gR = Quaternion(ego_pose['rotation']).rotation_matrix


{'lane_divider': [<shapely.geometry.linestring.LineString object at 0x2b6d0ddfc100>, <shapely.geometry.linestring.LineString object at 0x2b6d0ddfc5b0>, <shapely.geometry.linestring.LineString object at 0x2b6d0ddfcb50>, <shapely.geometry.linestring.LineString object at 0x2b6d0ddfcc70>, <shapely.geometry.linestring.LineString object at 0x2b6d0ddfc190>, <shapely.geometry.linestring.LineString object at 0x2b6d0ddfc4c0>, <shapely.geometry.linestring.LineString object at 0x2b6d0ddfcfd0>, <shapely.geometry.linestring.LineString object at 0x2b6d9dc59ac0>, <shapely.geometry.linestring.LineString object at 0x2b6d02e34fa0>, <shapely.geometry.linestring.LineString object at 0x2b6d9dc59ee0>], 'lane': [<shapely.geometry.multipolygon.MultiPolygon object at 0x2b6d0ddfc070>, <shapely.geometry.multipolygon.MultiPolygon object at 0x2b6d0ddfcfa0>, <shapely.geometry.multipolygon.MultiPolygon object at 0x2b6d02e34f70>, <shapely.geometry.multipolygon.MultiPolygon object at 0x2b6de9c06a30>, <shapely.geometry.

/tmp/ipykernel_34154/3560806371.py:24: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gt_list = nusc_map(((*center_xy,*size_xy), yaw, location))
/tmp/ipykernel_34154/2320943453.py:117: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for _g in g:
/home/zhenglt/mmdev11/detr3d-dev11/projects/detr3d/vis_zlt.py:150: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(instances_3d['gt_labels_3d']).to(device)


n008-2018-09-18-14-35-12-0400__LIDAR_TOP__1537295816397605
reference torch.Size([2169, 3])
900 1600
torch.Size([1, 2168, 3])
frame: 30925


In [11]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.patches import Arrow, Rectangle
from matplotlib.patches import Polygon as mPolygon
from nuscenes.map_expansion.bitmap import BitMap
from nuscenes.map_expansion.map_api import NuScenesMapExplorer
from shapely import affinity
from shapely.geometry import Polygon, MultiPolygon, LineString, Point, box
from typing import Dict, List, Optional, Tuple, Union
from nuscenes.map_expansion.map_api import NuScenesMap, NuScenesMapExplorer
from nuscenes.eval.common.utils import quaternion_yaw, Quaternion


class VectorizeLocalMap(object):

    def __init__(self,
                 data_root="./datasets/nuScenes/",
                 line_classes=['road_divider', 'lane_divider'],
                 ped_crossing_classes=['ped_crossing'],
                 contour_classes=['road_segment', 'lane'],
                 **kwargs):
        '''
        Args:
            fixed_num = -1 : no fixed num
        '''
        super().__init__()
        self.data_root = data_root
        self.MAPS = ['boston-seaport', 'singapore-hollandvillage',
                     'singapore-onenorth', 'singapore-queenstown']
        self.line_classes = line_classes
        self.ped_crossing_classes = ped_crossing_classes
        self.contour_classes = contour_classes

        self.nusc_maps = {}
        self.map_explorer = {}
        for loc in self.MAPS:
            self.nusc_maps[loc] = NuScenesMap(
                dataroot=self.data_root, map_name=loc)
            self.map_explorer[loc] = NuScenesMapExplorer(self.nusc_maps[loc])

    def retrive_geom(self, patch_params):
        '''
            Get the geometric data.
            Returns: dict
        '''
        patch_box, patch_angle, location = patch_params
        geoms_dict = {}
        centerline_records = []

        layers = \
            self.line_classes + self.ped_crossing_classes + \
            self.contour_classes

        layers = set(layers)
        for layer_name in layers:

            return_token = False

            # retrive the geo
            if layer_name in self.nusc_maps[location].non_geometric_line_layers:
                geoms = self.map_explorer[location]._get_layer_line(
                    patch_box, patch_angle, layer_name)
            elif layer_name in self.nusc_maps[location].lookup_polygon_layers:
                geoms = self.map_explorer[location]._get_layer_polygon(
                    patch_box, patch_angle, layer_name)
            else:
                raise ValueError('{} is not a valid layer'.format(layer_name))

            if geoms is None:
                continue

            # change every geoms set to list
            if not isinstance(geoms, list):
                geoms = [geoms, ]

            geoms_dict[layer_name] = geoms

        return geoms_dict

    # def get_global_patch(self, input_dict: dict):
    #     # transform to global coordination
    #     location = input_dict['location']
    #     translation = input_dict['ego2global_translation']
    #     rotation = Quaternion(
    #         input_dict['ego2global_rotation']).rotation_matrix
    #     lidar2ego_translation = input_dict['lidar2ego_translation']
    #     lidar2ego_rotation = Quaternion(
    #         input_dict['lidar2ego_rotation']).rotation_matrix

    #     rotation = rotation @ lidar2ego_rotation  # [3, 3]patch_box
    #     patch_box = (map_pose[0], map_pose[1],
    #                  self.patch_size[0], self.patch_size[1])
    #     patch_angle = quaternion_yaw(rotation) / np.pi * 180

    #     patch_params = (patch_box, patch_angle, location)
    #     return patch_params

    def __call__(self, patch_params):

        geoms_dict = self.retrive_geom(patch_params)
        print(geoms_dict)
        def sample_pts(pts):
            line = LineString(pts)
            distances = list(np.arange(1.0,
                                       line.length, 1.0))
            sampled_points = np.array([list(line.interpolate(distance).coords)
                                       for distance in distances]).reshape(-1, 2)
            return sampled_points

        kpts = []
        for k in geoms_dict:
            for g in geoms_dict[k]:
                if 'Polygon' in g.type:
                    if 'Multi' in g.type:
                        for _g in g:
                            kpts.append(np.array(_g.exterior))
                            kpts[-1] = sample_pts(kpts[-1])
                    else:
                        kpts.append(np.array(g.exterior))
                        kpts[-1] = sample_pts(kpts[-1])
                else:
                    kpts.append(np.array(g))
                    kpts[-1] = sample_pts(kpts[-1])

        return kpts


class CNuScenesMapExplorer(NuScenesMapExplorer):
    def __ini__(self, *args, **kwargs):
        super(self, CNuScenesMapExplorer).__init__(*args, **kwargs)

    def render_map_patch(self,
                         box_coords: Tuple[float, float, float, float],
                         layer_names: List[str] = None,
                         alpha: float = 0.5,
                         figsize: Tuple[float, float] = (15, 15),
                         render_egoposes_range: bool = True,
                         render_legend: bool = True,
                         bitmap: Optional[BitMap] = None) -> Tuple[Figure, Axes]:
        """
        Renders a rectangular patch specified by `box_coords`. By default renders all layers.
        :param box_coords: The rectangular patch coordinates (x_min, y_min, x_max, y_max).
        :param layer_names: All the non geometric layers that we want to render.
        :param alpha: The opacity of each layer.
        :param figsize: Size of the whole figure.
        :param render_egoposes_range: Whether to render a rectangle around all ego poses.
        :param render_legend: Whether to render the legend of map layers.
        :param bitmap: Optional BitMap object to render below the other map layers.
        :return: The matplotlib figure and axes of the rendered layers.
        """
        x_min, y_min, x_max, y_max = box_coords.bounds

        if layer_names is None:
            layer_names = self.map_api.non_geometric_layers

        fig = plt.figure(figsize=figsize)

        local_width = x_max - x_min
        local_height = y_max - y_min
        assert local_height > 0, 'Error: Map patch has 0 height!'
        local_aspect_ratio = local_width / local_height

        ax = fig.add_axes([0, 0, 1, 1 / local_aspect_ratio])

        if bitmap is not None:
            bitmap.render(self.map_api.canvas_edge, ax)

        for layer_name in layer_names:
            self._render_layer(ax, layer_name, alpha)

        x_margin = np.minimum(local_width / 4, 50)
        y_margin = np.minimum(local_height / 4, 10)
        ax.set_xlim(x_min - x_margin, x_max + x_margin)
        ax.set_ylim(y_min - y_margin, y_max + y_margin)

        if render_egoposes_range:
            bbox = np.array(box_coords.exterior.coords)
            WH = np.linalg.norm(bbox[1:3]-bbox[:2], axis=-1)
            W, iw = max(WH), np.argmax(WH)
            H, ih = min(WH), np.argmin(WH)

            ax.add_patch(mPolygon(np.array(box_coords.exterior.coords), fill=False, linestyle='-.', color='red',
                                  lw=2))
            ax.text(bbox[ih:2+ih, 0].mean(), bbox[ih:2+ih, 1].mean(), "%g m" % H,
                    fontsize=14, weight='bold')
            ax.text(bbox[iw:2+iw, 0].mean(), bbox[iw:2+iw, 1].mean(), "%g m" % W,
                    fontsize=14, weight='bold')

        if render_legend:
            ax.legend(frameon=True, loc='upper right')

        return fig, ax

    def _get_layer_polygon(self,
                           patch_box: Tuple[float, float, float, float],
                           patch_angle: float,
                           layer_name: str,
                           return_token: bool = False) -> Tuple[List[Polygon], Optional[List[str]]]:
        """
         Retrieve the polygons of a particular layer within the specified patch.
         :param patch_box: Patch box defined as [x_center, y_center, height, width].
         :param patch_angle: Patch orientation in degrees.
         :param layer_name: name of map layer to be extracted.
         :return: List of Polygon in a patch box.
         """
        if layer_name not in self.map_api.non_geometric_polygon_layers+['lane_connector']:
            raise ValueError('{} is not a polygonal layer'.format(layer_name))

        patch_x = patch_box[0]
        patch_y = patch_box[1]

        patch = self.get_patch_coord(patch_box, patch_angle)

        records = getattr(self.map_api, layer_name)

        polygon_list = []
        polygon_tokens_list = []
        valid_records = []
        if layer_name == 'drivable_area':
            for record in records:

                for polygon_token in record['polygon_tokens']:

                    polygon = self.map_api.extract_polygon(polygon_token)
                    new_polygon = polygon.intersection(patch)

                    if not new_polygon.is_empty:
                        new_polygon = affinity.rotate(new_polygon, -patch_angle,
                                                      origin=(patch_x, patch_y), use_radians=False)
                        new_polygon = affinity.affine_transform(new_polygon,
                                                                [1.0, 0.0, 0.0, 1.0, -patch_x, -patch_y])
                        if new_polygon.geom_type is 'Polygon':
                            new_polygon = MultiPolygon([new_polygon])
                        polygon_list.append(new_polygon)
                        polygon_tokens_list.append(polygon_token)
        else:
            for record in records:
                if record['polygon_token'] is None:
                    # import ipdb
                    # ipdb.set_trace()
                    continue
                polygon = self.map_api.extract_polygon(record['polygon_token'])

                # if polygon.intersects(patch) or polygon.within(patch):
                #     if not polygon.is_valid:
                #         print('within: {}, intersect: {}'.format(polygon.within(patch), polygon.intersects(patch)))
                #         print('polygon token {} is_valid: {}'.format(record['polygon_token'], polygon.is_valid))

                # polygon = polygon.buffer(0)

                if polygon.is_valid:
                    # if within or intersect :

                    new_polygon = polygon.intersection(patch)
                    # new_polygon = polygon

                    if not new_polygon.is_empty:
                        new_polygon = affinity.rotate(new_polygon, -patch_angle,
                                                      origin=(patch_x, patch_y), use_radians=False)
                        new_polygon = affinity.affine_transform(new_polygon,
                                                                [1.0, 0.0, 0.0, 1.0, -patch_x, -patch_y])
                        if new_polygon.geom_type is 'Polygon':
                            new_polygon = MultiPolygon([new_polygon])
                        polygon_list.append(new_polygon)
                        polygon_tokens_list.append(record['polygon_token'])
                        # print('polygon token accepeted:', record['polygon_token'])
                        if layer_name in '(lane,road_block)':
                            record['from_edge_line'] = \
                                to_patch_coord(self.map_api.extract_line(
                                    record['from_edge_line_token']), patch_angle, patch_x, patch_y)
                            record['to_edge_line'] = \
                                to_patch_coord(self.map_api.extract_line(
                                    record['to_edge_line_token']), patch_angle, patch_x, patch_y)

                        if layer_name in ['lane', 'lane_connector']:
                            centerline = self.map_api.discretize_lanes(
                                record, 0.5)
                            centerline = list(self.map_api.discretize_lanes(
                                [record['token']], 0.5).values())[0]
                            centerline = LineString(
                                np.array(centerline)[:, :2])
                            centerline = centerline.intersection(patch)
                            record['centerline'] = \
                                to_patch_coord(
                                    centerline, patch_angle, patch_x, patch_y)

                        valid_records.append(record)

        if return_token:
            return polygon_list, polygon_tokens_list, valid_records

        return polygon_list

    def _get_layer_line(self,
                        patch_box: Tuple[float, float, float, float],
                        patch_angle: float,
                        layer_name: str,
                        return_token: bool = False) -> Optional[List[LineString]]:
        """
        Retrieve the lines of a particular layer within the specified patch.
        :param patch_box: Patch box defined as [x_center, y_center, height, width].
        :param patch_angle: Patch orientation in degrees.
        :param layer_name: name of map layer to be converted to binary map mask patch.
        :return: List of LineString in a patch box.
        """
        if layer_name not in self.map_api.non_geometric_line_layers:
            raise ValueError("{} is not a line layer".format(layer_name))

        if layer_name is 'traffic_light':
            return None

        patch_x = patch_box[0]
        patch_y = patch_box[1]

        patch = self.get_patch_coord(patch_box, patch_angle)

        line_list = []
        line_tokens_list = []
        valid_records = []
        records = getattr(self.map_api, layer_name)
        for record in records:
            line = self.map_api.extract_line(record['line_token'])
            if line.is_empty:  # Skip lines without nodes.
                continue

            new_line = line.intersection(patch)
            if not new_line.is_empty:
                new_line = affinity.rotate(
                    new_line, -patch_angle, origin=(patch_x, patch_y), use_radians=False)
                new_line = affinity.affine_transform(new_line,
                                                     [1.0, 0.0, 0.0, 1.0, -patch_x, -patch_y])
                line_list.append(new_line)
                line_tokens_list.append(record['line_token'])
                valid_records.append(record)

        if return_token:
            return line_list, line_tokens_list, valid_records

        return line_list


def to_patch_coord(new_polygon, patch_angle, patch_x, patch_y):
    new_polygon = affinity.rotate(new_polygon, -patch_angle,
                                  origin=(patch_x, patch_y), use_radians=False)
    new_polygon = affinity.affine_transform(new_polygon,
                                            [1.0, 0.0, 0.0, 1.0, -patch_x, -patch_y])
    return new_polygon


# utils
def plot(polygon, buffer=1):
    from descartes.patch import PolygonPath, PolygonPatch
    fig, ax = plt.subplots()
    ax.add_patch(PolygonPatch(polygon.buffer(buffer),
                 fill=False, fc='orange', ec='blue', alpha=0.5))
    xy = np.array(polygon.buffer(buffer).exterior.coords)
    ax.plot(xy[:, 0], xy[:, 1], 'o', color='orange')

    ax.add_patch(PolygonPatch(polygon, fill=False,
                 fc='red', ec='red', alpha=0.5))
    xy = np.array(polygon.exterior.coords)
    ax.plot(xy[:, 0], xy[:, 1], 'o', color='red')
    fig.savefig('test_buffer.png', dpi=200)


<>:234: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:264: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:311: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:234: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:264: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:311: SyntaxWarning: "is" with a literal. Did you mean "=="?
/tmp/ipykernel_34154/2320943453.py:234: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if new_polygon.geom_type is 'Polygon':
/tmp/ipykernel_34154/2320943453.py:264: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if new_polygon.geom_type is 'Polygon':
/tmp/ipykernel_34154/2320943453.py:311: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if layer_name is 'traffic_light':


In [50]:
pad_size = 3
class VectorizeLocalMap_new(object):

    def __init__(self,
                 data_root="./datasets/nuScenes/",
                 line_classes=['road_divider', 'lane_divider'],
                 ped_crossing_classes=['ped_crossing'],
                 contour_classes=['road_segment', 'lane'],
                 **kwargs):
        '''
        Args:
            fixed_num = -1 : no fixed num
        '''
        super().__init__()
        self.data_root = data_root
        self.MAPS = ['boston-seaport', 'singapore-hollandvillage',
                     'singapore-onenorth', 'singapore-queenstown']
        self.line_classes = line_classes
        self.ped_crossing_classes = ped_crossing_classes
        self.contour_classes = contour_classes
        self.patch_size = [100,100]
        self.nusc_maps = {}
        self.map_explorer = {}
        for loc in self.MAPS:
            self.nusc_maps[loc] = NuScenesMap(
                dataroot=self.data_root, map_name=loc)
            self.map_explorer[loc] = NuScenesMapExplorer(self.nusc_maps[loc])
    
    def get_global_patch(self, input_dict: dict):
        # transform to global coordination
        location = input_dict['location']
        translation = input_dict['ego2global_translation']
        rotation = Quaternion(input_dict['ego2global_rotation']).rotation_matrix

        map_pose = translation[:2]
        rotation = Quaternion(matrix=rotation)
        patch_box = (map_pose[0], map_pose[1],
                     self.patch_size[0]+pad_size, self.patch_size[1]+pad_size)
        patch_angle = quaternion_yaw(rotation) / np.pi * 180

        patch_params = (patch_box, patch_angle, location)

        return patch_params
    def filter_pts_inrange(self,pts):
        max_x = self.patch_size[0] / 2
        max_y = self.patch_size[1] / 2
        mask = (-max_x < pts[...,0]) & (pts[..., 0] < max_x) & \
               (-max_y < pts[...,1]) & (pts[..., 1] < max_y)
        pts = pts[mask]
        # return none if nothing
        return pts

    def ego2lidar_and_filter(self, pts, input_dict, return_dim = 3):
        
        if 'l2e_r' in input_dict:
            l2e_R = input_dict['l2e_r']
            l2e_t = input_dict['l2e_t']
        else:
            l2e_R = Quaternion(input_dict['lidar2ego_rotation']).rotation_matrix
            l2e_t = input_dict['lidar2ego_translation']
        l2e = np.identity(4)
        l2e[:3,:3] = np.array(l2e_R)
        l2e[:3,3] = np.array(l2e_t)
        # e2g_t = input_dict['ego2global_translation']
        e2g_R = Quaternion(input_dict['ego2global_rotation'])
        pts = torch.tensor(pts)
        pts_z = torch.zeros_like(pts[:,0:1])
        pts_t = torch.ones_like(pts[:,0:1])
        pts = torch.cat([pts, pts_z, pts_t],dim=-1)
        e2g_conpensate = Quaternion(axis=[0, 1, 0], radians=e2g_R.yaw_pitch_roll[1]) * \
                         Quaternion(axis=[1, 0, 0], radians=e2g_R.yaw_pitch_roll[2])
        e2g_pitch_roll = e2g_conpensate.rotation_matrix
        g2e_pitch_roll = np.linalg.inv(e2g_pitch_roll)
        g2e_pitch_roll_4d = np.identity(4)
        g2e_pitch_roll_4d[:3,:3] = g2e_pitch_roll
        pts = torch.matmul(torch.tensor(g2e_pitch_roll_4d),pts.T).T
        pts = torch.matmul(e2l,pts.T).T
        pts = self.filter_pts_inrange(pts)
        return np.array(pts[:,:return_dim])

    def retrive_geom(self, patch_params):
        '''
            Get the geometric data.
            Returns: dict
        '''
        patch_box, patch_angle, location = patch_params
        geoms_dict = {}
        centerline_records = []

        layers = \
            self.line_classes + self.ped_crossing_classes + \
            self.contour_classes

        layers = set(layers)
        for layer_name in layers:

            return_token = False

            # retrive the geo
            if layer_name in self.nusc_maps[location].non_geometric_line_layers:
                geoms = self.map_explorer[location]._get_layer_line(
                    patch_box, patch_angle, layer_name)
            elif layer_name in self.nusc_maps[location].lookup_polygon_layers:
                geoms = self.map_explorer[location]._get_layer_polygon(
                    patch_box, patch_angle, layer_name)
            else:
                raise ValueError('{} is not a valid layer'.format(layer_name))

            if geoms is None:
                continue

            # change every geoms set to list
            if not isinstance(geoms, list):
                geoms = [geoms, ]

            geoms_dict[layer_name] = geoms

        return geoms_dict

    def __call__(self, input_dict):
        print('new')
        patch_params = self.get_global_patch(input_dict)
        geoms_dict = self.retrive_geom(patch_params)

        def sample_pts(pts):
            line = LineString(pts)
            distances = list(np.arange(1.0,
                                       line.length, 1.0))
            sampled_points = np.array([list(line.interpolate(distance).coords)
                                       for distance in distances]).reshape(-1, 2)
            sampled_points = self.ego2lidar_and_filter(sampled_points,input_dict)
            return sampled_points

        kpts = []
        for k in geoms_dict:
            for g in geoms_dict[k]:
                if 'Polygon' in g.type:
                    if 'Multi' in g.type:
                        for _g in g:
                            kpts.append(np.array(_g.exterior))
                            kpts[-1] = sample_pts(kpts[-1])
                    else:
                        kpts.append(np.array(g.exterior))
                        kpts[-1] = sample_pts(kpts[-1])
                else:
                    kpts.append(np.array(g))
                    kpts[-1] = sample_pts(kpts[-1])

        return kpts

In [51]:
nusc_map = VectorizeLocalMap_new(data_root='data/nuscenes/')
map_data = pandas.read_pickle('debug/lyc_map/map_data_upsampling.pkl')
for token in map_data:
    idx = token2idx[token]
    # if token != 'a08cf86f5b2a4f5abdee5756820aa66f':
    #     continue
    frame = dataset[idx]
    from pyquaternion import Quaternion
    e2g = np.array(frame['data_samples'].metainfo['ego2global'])
    rot = Quaternion(matrix=e2g[:3,:3],atol=1e-5)
    l2e = np.array(frame['data_samples'].metainfo['lidar2ego'])
    location = frame['data_samples'].metainfo['city_name']
    input_dict = {
        'l2e_r': l2e[:3,:3].tolist(),
        'l2e_t': l2e[:3,3].tolist(),
        'ego2global_rotation': rot.q,
        'ego2global_translation': e2g[:2,3].tolist(),
        'location': location
    }
    gt_list = nusc_map(input_dict)
    l2e = np.array(frame['data_samples'].metainfo['lidar2ego'])
    map_points = []
    for gt in gt_list:
        map_points.append(torch.tensor(gt))
    map_points = torch.cat(map_points)
    # print(map_points)
    view_frame(token, map_points[:,:3])


new


/tmp/ipykernel_34154/3294748437.py:20: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  gt_list = nusc_map(input_dict)
/tmp/ipykernel_34154/2638741340.py:139: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for _g in g:
/home/zhenglt/mmdev11/detr3d-dev11/projects/detr3d/vis_zlt.py:150: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(instances_3d['gt_labels_3d']).to(device)


n008-2018-09-18-14-35-12-0400__LIDAR_TOP__1537295816397605
reference torch.Size([1900, 3])
900 1600
torch.Size([1, 1899, 3])
frame: 30925
new
n008-2018-08-30-15-31-50-0400__LIDAR_TOP__1535657803550545
reference torch.Size([2630, 3])
900 1600
torch.Size([1, 2629, 3])
frame: 31669
new
n015-2018-09-25-13-17-43+0800__LIDAR_TOP__1537853040648000
reference torch.Size([818, 3])
900 1600
torch.Size([1, 817, 3])
frame: 33519
new
n015-2018-11-21-19-21-35+0800__LIDAR_TOP__1542799429797267
reference torch.Size([616, 3])
900 1600
torch.Size([1, 615, 3])
frame: 33683
new
n015-2018-11-21-19-21-35+0800__LIDAR_TOP__1542799417949014
reference torch.Size([579, 3])
900 1600
torch.Size([1, 578, 3])
frame: 33659
new
n008-2018-09-18-14-35-12-0400__LIDAR_TOP__1537295962798292
reference torch.Size([1419, 3])
900 1600
torch.Size([1, 1418, 3])
frame: 31113
new
n015-2018-10-08-15-44-23+0800__LIDAR_TOP__1538985027397601
reference torch.Size([661, 3])
900 1600
torch.Size([1, 660, 3])
frame: 33137
new
n015-2018-10-0

KeyboardInterrupt: 